## 임베딩 읽어들이기

In [1]:
def load_word_embeddings(path, num_vecs=30000):
    embeddings, vocab = [], []
    f = open(path, "r", encoding="utf-8").readlines()[1:num_vecs + 1]
    for line in f:
        splited_line = line.strip().split(" ")
        word = splited_line[0]
        vec = [float(el) for el in splited_line[1:]]
        vocab.append(word)
        embeddings.append(vec)
    word_to_index = {word: idx for idx, word in enumerate(vocab)}
    index_to_word = {idx: word for idx, word in enumerate(vocab)}
    vocab_size = len(vocab)
    return embeddings, vocab, word_to_index, index_to_word, vocab_size

In [2]:
embeddings, vocab, word_to_index, index_to_word, vocab_size = load_word_embeddings("embedding/word2vec.txt")

In [7]:
# word_to_index['폰']
embeddings[word_to_index['폰']]

[0.40756693,
 0.10399851,
 0.068960644,
 -0.06728446,
 -0.04627618,
 0.11270078,
 -0.5182983,
 0.010880889,
 0.26014423,
 0.12969808,
 0.131198,
 -0.12764914,
 0.5413734,
 -0.5105737,
 0.2966412,
 -0.21341276,
 0.42202273,
 0.39021936,
 -0.24918261,
 -0.21797988,
 -0.038810458,
 0.19363868,
 0.33906236,
 -0.028319051,
 0.016351104,
 0.104889855,
 -0.22713748,
 -0.1870471,
 0.10869626,
 -0.025511548,
 0.76009697,
 0.11837887,
 0.69456154,
 0.1168128,
 -0.033387233,
 -0.04071568,
 0.39109078,
 -0.277839,
 0.48387015,
 0.17641912,
 0.23689614,
 0.048578106,
 -0.33016205,
 0.021788016,
 0.12018698,
 -0.5191495,
 -0.034779374,
 0.20541409,
 0.47657725,
 0.15663806,
 -0.86120975,
 0.025745058,
 0.45544395,
 0.10810242,
 0.2953998,
 -0.59026104,
 0.2927954,
 -0.39751402,
 -0.7756063,
 -0.35340938,
 -0.08575064,
 0.054851316,
 0.005910598,
 -0.018256905,
 0.02538809,
 -0.23107912,
 0.31236434,
 -0.07843713,
 -0.10905662,
 -1.1090585,
 0.25013113,
 -0.293665,
 0.41080025,
 0.2516035,
 -0.067625

## 관심 기능 정의

In [8]:
functions = ["배터리", "촬영", "디자인", "화면", "OS", "색"]
function_size = len(functions)

## 거리 행렬 구하기

In [9]:
import numpy as np
from scipy.spatial import distance

distance_matrix = np.zeros([function_size, vocab_size], dtype=np.float16)

In [10]:
for func_idx, func in enumerate(functions):
    func_vec = embeddings[word_to_index[func]]
    for word_idx, word_vec in enumerate(embeddings):
        distance_matrix[func_idx, word_idx] = distance.euclidean(func_vec, word_vec)

In [11]:
distance_matrix

array([[3.75 , 4.094, 3.812, ..., 3.172, 3.152, 3.168],
       [4.215, 4.484, 4.266, ..., 3.525, 3.707, 3.547],
       [3.682, 3.998, 3.674, ..., 2.922, 3.133, 3.115],
       [3.807, 4.19 , 3.81 , ..., 3.295, 3.426, 3.342],
       [3.82 , 4.258, 3.99 , ..., 3.201, 3.264, 3.162],
       [4.29 , 4.547, 4.25 , ..., 3.566, 3.541, 3.682]], dtype=float16)

## 가중치 행렬 구하기

In [12]:
# 정규화

weight_matrix = np.zeros([function_size, vocab_size], dtype=np.float16)
for func_idx, per_func_distances in enumerate(distance_matrix):
    values = np.exp(-(per_func_distances ** 2) / 15)
    mean = np.mean(values)
    weight_matrix[func_idx, :] = values - mean + 0.5

In [13]:
weight_matrix

array([[0.4458, 0.381 , 0.4338, ..., 0.5654, 0.57  , 0.5664],
       [0.4368, 0.3926, 0.428 , ..., 0.5674, 0.5312, 0.5635],
       [0.4202, 0.3596, 0.4216, ..., 0.581 , 0.535 , 0.5386],
       [0.4539, 0.3835, 0.4531, ..., 0.5586, 0.531 , 0.549 ],
       [0.4436, 0.3645, 0.4116, ..., 0.5703, 0.5576, 0.579 ],
       [0.4136, 0.372 , 0.42  , ..., 0.548 , 0.553 , 0.525 ]],
      dtype=float16)

## 가중치 행렬 시각화

In [14]:
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show

output_notebook()
def make_plot(title, hist, edges):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.y_range.start = 0
    p.grid.grid_line_color="white"
    return p

Loading BokehJS ...

In [15]:
plots = []
for func_idx, func in enumerate(functions):
    hist, edges = np.histogram(weight_matrix[func_idx], density=True, bins=50)
    plots.append(make_plot(func, hist, edges))
show(gridplot(plots, ncols=2, plot_width=400, plot_height=300, toolbar_location=None))

정규분포를 가운데 절반에서 자른 모양

In [16]:
battery_weights = [(token, score) for token, score in zip(vocab, weight_matrix[0])]

In [17]:
sorted(battery_weights, key=lambda x: x[1], reverse=True)

[('배터리', 1.055),
 ('밧데리', 0.9185),
 ('표준형', 0.7617),
 ('터리', 0.7246),
 ('대용량', 0.723),
 ('충전기', 0.703),
 ('러닝', 0.6924),
 ('슬림형', 0.673),
 ('발열', 0.6694),
 ('리튬', 0.668),
 ('상판', 0.664),
 ('폴리머', 0.664),
 ('대기전력', 0.6636),
 ('커넥터', 0.657),
 ('키트', 0.6553),
 ('급속', 0.655),
 ('간당간당', 0.655),
 ('기판', 0.6514),
 ('TTA', 0.649),
 ('회로', 0.6436),
 ('껍데기', 0.6426),
 ('오래간다', 0.6426),
 ('여분', 0.639),
 ('mAH', 0.639),
 ('나사', 0.6377),
 ('밧', 0.6357),
 ('접점', 0.635),
 ('연식', 0.6323),
 ('단다고', 0.6313),
 ('접지', 0.6304),
 ('극심', 0.6304),
 ('커버도', 0.6294),
 ('단다', 0.629),
 ('배불뚝이', 0.6284),
 ('RMAA', 0.6284),
 ('Ap', 0.6274),
 ('뽀대', 0.627),
 ('외장형', 0.627),
 ('런타임', 0.627),
 ('주범', 0.627),
 ('암페어', 0.6265),
 ('소모율', 0.6265),
 ('타폰', 0.6265),
 ('리시버', 0.6255),
 ('쿨러', 0.6255),
 ('놑', 0.625),
 ('가로길이', 0.625),
 ('ISP', 0.625),
 ('뒷부분', 0.623),
 ('뒤쪽', 0.623),
 ('짹', 0.623),
 ('tta', 0.623),
 ('교실', 0.6226),
 ('하판', 0.6226),
 ('에너지', 0.622),
 ('가품', 0.622),
 ('정확도', 0.6206),
 ('턱없이', 0.6206),
 ('옹이', 0

## Document-Term Matrix 구축

In [20]:
corpus = open("data/phone_review_mecab.txt", "r", encoding="utf-8").readlines()
corpus = list(set([el.strip() for el in corpus if len(el.split()) < 31]))

In [25]:
import random
corpus = random.sample(corpus, 3000)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(vocabulary=vocab
                             , binary=True # 해당 단어가 1 번 이상 쓰였으면 1
                             , ngram_range=(1, 1) # 한 단어씩 
                             , tokenizer=lambda x: x.strip().split()) # tokenizer 는 공백으로 split

In [27]:
dtm = vectorizer.fit_transform(corpus).toarray()
word_counts_per_doc = np.sum(dtm, axis=1) 
result = np.dot(dtm, weight_matrix.T)

In [28]:
# 문서 사이즈(말뭉치), vocab 사이즈
dtm.shape

(3000, 23599)

In [29]:
word_counts_per_doc

array([10,  9,  7, ..., 19,  2, 13], dtype=int64)

In [30]:
word_counts_per_doc ** 0.6

array([3.98107171, 3.73719282, 3.21409585, ..., 5.85129724, 1.51571657,
       4.65978642])

## 최종 결과물 도출

In [31]:
for func_idx, func in enumerate(functions):
    print("##### " + func)

    # 아래에 (-) 부호 붙여주는 이유는 내림차순으로 보기위한 트릭
    scores = - result[:, func_idx] / (word_counts_per_doc ** 0.6) # 문장을 구성하는 단어 수로 나눠주어서 단어 갯수 때문에 영향받지 않게 한다, 0.6 을 제곱해준건 smoothing 해준 것
    for sent_idx in scores.argsort()[:10]:
        # 원래 데이터 그대로 보기 위해 다시 (-) 부호 붙여준다
        print(corpus[sent_idx].strip(), - scores[sent_idx])
    print("\n")

##### 배터리
오메 잠바 주머니 에 넣 어 놨더니 뜨끈뜨끈 ㅋㅋ 손난로 인데요 허허 제 미라크 가 문제 인가요 아이폰 은 그렇게 따뜻 하 지 않 은데 미라크 가 이상 해요 후후 1.5756941888207259
그렇 군요 다만 너무 비싼 가격 과 화면 크기 작음 과 배터리 내장 이 라서 이 3 가지 만 따져도 갤 s 3 승리 전 옵 이 이 유저 임다 1.556766978112604
이건 옴니아 2 윈 모지 우고 안드로이드 깔 아도 이거 랑 비슷 할 텐데 ㅋ 이제 하드웨어 는 고만고만 하 고 이제 소프트웨어 로 평가 가 갈리 네요 1.5434776656613163
시리우스 가 감 악식 이 라 습도 가 높 은 장마철 에 터치 패널 과 액정 이 들러붙 어서 터치 가 고자 가 되 는 종 특이 있 었 습니다 1.5134640131302755
일단 미디어 데 이때 겔 럭 시 노트 ap 알려 주 는 부분 에서 1 5 듀얼 코어 로 끝냈 다는 게 문제 1 0 초 정도 설명 했 나 1.4961488851467812
조금 늦 게 나와도 발열 과 배터리 만 해결 되 서 나오 면 후면 배꼽 디자인 은 애교 로 봐 줘야 겠 네요 ㅋㅋ 1.4874967409968187
저 도 2 년 7 달째 사용 중 고장 도 안 나 고 다만 배터리 가 6 시간 정품 큰 거 쓰 는데 ㅠ 1.487016430207233
다행히 부품 수급 문제 로 바뀐 텀블러 디자인 이 1 차 보다 개인 적 기준 에서 는 맘 에 안 든다 는 아니 고 암튼 그래서 좀 다행 이 네요 1.472918686659656
저 는 하나 만 교환 받 고 하나 는 못 받 아서 작년 말 에 대용량 배터리 1 개 도 구매 해서 3 개 씁니다 1.4695747770908933
씨 피 유부 터 옵 쥐 가 더 좋 아요 카메라 배터리 외장 메모리 sd 카드 로 까이 는 거 지 1.4681602509558895


##### 촬영
오메 잠바 주머니 에 넣 어 놨더니 뜨끈뜨끈 ㅋㅋ 손난로 인데요 허허 제 미라크 가 문제 인가요 아이폰 은 그렇게 따

C:\Users\Minju\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
